<a href="https://colab.research.google.com/github/PEA-IA-DS-Proyecto-1/HomePricePredictorBCN/blob/main/DecisionTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importar librerias y DataSet
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

url = 'https://raw.githubusercontent.com/PEA-IA-DS-Proyecto-1/HomePricePredictorBCN/main/Barcelona_Fotocasa_HousingPrices.csv'
df = pd.read_csv(url)

In [ ]:
id_column = df.iloc[:, 0]
num_cols = df.select_dtypes(include=['number']).drop(columns=[id_column.name], errors='ignore').columns.tolist()
matriz_corr = df[num_cols].corr()
plt.figure(figsize=(10, 8))
plt.matshow(matriz_corr, cmap='coolwarm', vmin=-1, vmax=1, fignum=1)
plt.colorbar()
plt.xticks(range(len(num_cols)), num_cols, rotation=45, ha='left')
plt.yticks(range(len(num_cols)), num_cols)
plt.title('Matriz de correlación', pad=20)
for i in range(len(num_cols)):
    for j in range(len(num_cols)):
        plt.text(j, i, f'{matriz_corr.iloc[i, j]:.2f}', ha='center', va='center', color='black')
plt.show()

In [ ]:
# Convertir la variables categóricas en variables dummy
df = pd.get_dummies(df, columns=['neighborhood', 'lift', 'terrace', 'real_state'], drop_first=True)

# Definir las variables independientes (X) y la variable dependiente (y)
X = df.drop(columns=['price'])
y = df['price']

# Crear y entrenar el modelo de regresión lineal
model = LinearRegression()
model.fit(X, y)

# Obtener los coeficientes del modelo
coefficients = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
print(coefficients)

In [ ]:
# Agrupa por barrio y calcula la media del precio
precio_por_barrio = df.groupby('neighborhood')['price'].mean()

# Ordena los resultados por precio en orden descendente
precio_por_barrio_ordenado = precio_por_barrio.sort_values(ascending=False)

# Muestra los resultados ordenados
print(precio_por_barrio_ordenado)

In [ ]:
# Agrupa por barrio y calcula la media del precio
precio_por_barrio = df.groupby('real_state')['price'].mean()

# Ordena los resultados por precio en orden descendente
precio_por_barrio_ordenado = precio_por_barrio.sort_values(ascending=False)

# Muestra los resultados ordenados
print(precio_por_barrio_ordenado)

In [ ]:
df.columns
# Agrupa por barrio y calcula la media del precio
#precio_por_barrio = df.groupby('real_state')['price'].mean()

In [ ]:
# Agrupa por barrio y calcula la media del precio
precio_por_barrio = df.groupby('neighborhood')['square_meters'].mean()

# Ordena los resultados por precio en orden descendente
precio_por_barrio_ordenado = precio_por_barrio.sort_values(ascending=False)

# Muestra los resultados ordenados
print(precio_por_barrio_ordenado)

In [ ]:
precio_metros_por_barrio = df.groupby('neighborhood').agg({'price': 'mean', 'square_meters': 'mean'})

# Ordena por precio en orden descendente
precio_metros_por_barrio_ordenado = precio_metros_por_barrio.sort_values(by=['price'], ascending=False)

print(precio_metros_por_barrio_ordenado)

In [ ]:
# Agrupa por barrio y calcula la media del precio
precio_por_barrio = df.groupby('real_state')['square_meters'].mean()

# Ordena los resultados por precio en orden descendente
precio_por_barrio_ordenado = precio_por_barrio.sort_values(ascending=False)

# Muestra los resultados ordenados
print(precio_por_barrio_ordenado)

In [ ]:
# Cálculo del valor medio de metros cuadrados para cada tipo de vivienda en cada barrio
mean_square_meters = df.groupby(['neighborhood', 'real_state'])['square_meters'].mean().reset_index()

# Renombrar columna para evitar conflictos
mean_square_meters.rename(columns={'square_meters': 'mean_square_meters'}, inplace=True)

# Función para encontrar el tipo de vivienda más cercano basado en el valor medio de metros cuadrados
def find_closest_real_state(row):
    if pd.isna(row['real_state']):
        # Filtrado del DataFrame de medias para obtener el tipo de vivienda con el valor medio más cercano
        neighborhood_data = mean_square_meters[mean_square_meters['neighborhood'] == row['neighborhood']]
        closest_real_state = neighborhood_data.iloc[(neighborhood_data['mean_square_meters'] - row['square_meters']).abs().argsort()[:1]]['real_state'].values[0]
        return closest_real_state
    else:
        return row['real_state']

# Aplicar función y listar los resultados
df['predicted_real_state'] = df.apply(find_closest_real_state, axis=1)

# Filtrar de los registros con NaN en el campo real_state
result = df[pd.isna(df['real_state'])][['neighborhood', 'square_meters', 'predicted_real_state']]

print(result)


In [ ]:
# Averigüa cual es el valor medio de metros cuadrados para cada tipo de vivienda en cada barrio y en virtud a ese valor medio
# determina que tipo de vivienda es y otorga el tipo de vivienda como valor a cada registro que tenga Nan
# en el campo real_state.

# Primero, calculamos el valor medio de metros cuadrados para cada tipo de vivienda en cada barrio
# mean_square_meters = df.groupby(['neighborhood', 'real_state'])['square_meters'].mean().reset_index()

# Renombramos la columna para evitar conflictos
# mean_square_meters.rename(columns={'square_meters': 'mean_square_meters'}, inplace=True)

# Unimos esta información con el DataFrame original
# df = df.merge(mean_square_meters, on=['neighborhood', 'real_state'], how='left')

# Ahora, llenamos los NaN en el campo real_state
# def fill_real_state(row):
#     if pd.isna(row['real_state']):
#         # Filtramos el DataFrame de medias para obtener el tipo de vivienda con el valor medio más cercano
#         neighborhood_data = mean_square_meters[mean_square_meters['neighborhood'] == row['neighborhood']]
#         closest_real_state = neighborhood_data.iloc[(neighborhood_data['mean_square_meters'] - row['square_meters']).abs().argsort()[:1]]['real_state'].values[0]
#         return closest_real_state
#     else:
#         return row['real_state']

# df['real_state'] = df.apply(fill_real_state, axis=1)

# Eliminamos la columna de medias ya que no la necesitamos más
# df.drop(columns=['mean_square_meters'], inplace=True)

# print(df)
